In [1]:
import cv2
import os

input_folder = r"C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\Data\images1"
output_video = 'imvideo.mp4'

frame_rate = 1
width = 640
height = 480

# Use 'X264' for H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'X264')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))
c=0
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(input_folder, filename))

        img = cv2.resize(img, (width, height))

        video_writer.write(img)
        c+=1
    if c>=5:
        break 

video_writer.release()

In [2]:
import cv2
import os

input_folder = r"C:\Users\babag\PROJECT_FOLDERS\2_2PS\new for unet\Data\Data\masks1"
output_video = 'maskvideo.mp4'

frame_rate = 1
width = 640
height = 480

# Use 'X264' for H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'X264')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))
c=0
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(input_folder, filename))

        img = cv2.resize(img, (width, height))

        video_writer.write(img)
        c+=1
    if c>=5:
        break 

video_writer.release()

In [ ]:
video_file = 'imvideo.mp4'
cap = cv2.VideoCapture(video_file)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create a list to store the images
images = []

# Loop through each frame of the video
for i in range(total_frames):
    # Read the current frame
    ret, frame = cap.read()
    
    # If the frame was read successfully, convert it to a numpy array and add it to the list
    if ret:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        images.append(image)
    
    # If we've reached the end of the video, break out of the loop
    if i == total_frames - 1:
        break

# Release the video capture object
cap.release()

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
import numpy as np
import cv2

# Define dice coefficient and dice loss functions
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Load the model with custom objects
with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
    model = tf.keras.models.load_model("model.keras")

# Load and preprocess the image

video_file = 'imvideo.mp4'
cap = cv2.VideoCapture(video_file)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create a list to store the images
images = []

# Loop through each frame of the video
for i in range(total_frames):
    # Read the current frame
    ret, frame = cap.read()
    
    # If the frame was read successfully, convert it to a numpy array and add it to the list
    if ret:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        images.append(image)
    
    # If we've reached the end of the video, break out of the loop
    if i == total_frames - 1:
        break

# Release the video capture object
cap.release()

for i in range(len(images)):
    image = images[i]
    image = cv2.resize(image, (224, 224))       # [H, W, 3]
    x = image / 255.0                           # [H, W, 3]
    x = np.expand_dims(x, axis=0)               # [1, H, W, 3]

    # Predict the mask
    y_pred = model.predict(x, verbose=0)[0]     # [H, W, 1]
    y_pred = np.squeeze(y_pred, axis=-1)        # [H, W]
    y_pred = y_pred >= 0.5                      # Thresholding
    y_pred = y_pred.astype(np.uint8)            # Convert to uint8
    y_pred = np.expand_dims(y_pred, axis=-1)    # [H, W, 1]
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  # [H, W, 3]
    y_pred = y_pred * 255                       # Convert to 0-255 scale

    # Ensure both images are of the same type
    image = image.astype(np.uint8)

    # Overlay the mask on the original image
    overlay = cv2.addWeighted(image, 0.5, y_pred, 0.5, 0)

    # Save the result
    images[i]= overlay

    
    
output_video = 'predvideo.mp4'

frame_rate = 1
width = 640
height = 480

# Use 'X264' for H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'X264')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))


for i in images:
    img = i
    img = cv2.resize(img, (width, height))
    video_writer.write(img)
video_writer.release()




NameError: name 'output_path' is not defined